In [1]:
# LOCAL TEST
import requests

BASE_URL = "http://127.0.0.1:8000"

print(requests.get(f"{BASE_URL}/api/v1/healthcheck").json())

{'status': 'OK', 'timestamp': '2025-08-13T16:16:54.763493+00:00'}


In [9]:
# Evaluate
payload = {
    "user_message": "What is the capital of France?",
    "chat_history": [
        {"role": "user", "content": "Hi"},
        {"role": "assistant", "content": "The capital of Canada is Ottawa"},
    ],
}
resp = requests.post(f"{BASE_URL}/api/v1/evaluate", json=payload, timeout=60)


In [12]:
# APP TEST
from databricks.sdk.core import Config
import requests

config = Config(profile="DEFAULT")
token = config.oauth_token().access_token

session = requests.Session()
session.headers.update({
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json",
})

BASE_URL = "https://shm-chatbot-auth-test-delete-984752964297111.11.azure.databricksapps.com"

response = session.get(
    f"{BASE_URL}/api/v1/healthcheck", 
    allow_redirects=False)

print(response)

<Response [200]>
